## 모듈 import

In [1]:
#모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pycaret.regression  import *


# feature selection
from sklearn.linear_model import LinearRegression
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

#Scailing
from sklearn.preprocessing import PowerTransformer

#Label Encoding
from sklearn.preprocessing import LabelEncoder

In [2]:
#삡운드
import winsound as sd
# 삡싸운드
def beepsound():
    fr = 2000   # range : 37 ~ 32767
    du = 1000 # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

## data 불러오기

In [3]:
# 데이터가 저장된 경로 설정
data_path = '../data/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'train_df.csv', encoding='cp949')
test = pd.read_csv(data_path + 'test_df.csv', encoding='cp949')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [4]:
train

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3
...,...,...,...,...,...,...
31995,31995,5011001060063300,2635000026053400,식품,농산물,6
31996,31996,5011001095042400,1168000017002200,식품,기타식품,5
31997,31997,5011001108036200,4119700008012100,식품,농산물,9
31998,31998,5011001115011400,1132000015085100,식품,농산물,3


### 전처리

In [5]:
data = pd.concat([train,test]).reset_index(drop=True);data

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3.0
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3.0
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9.0
3,3,1154500002014200,5011000315087400,식품,농산물,10.0
4,4,1165000021008300,5011000177051200,식품,가공식품,3.0
...,...,...,...,...,...,...
36635,36635,5013000858004400,4725000719072200,식품,농산물,NaN
36636,36636,5013000870018300,2826000106075300,식품,농산물,NaN
36637,36637,5013000897086300,4311100034004300,식품,농산물,NaN
36638,36638,5013000902065100,4145000013011200,식품,농산물,NaN


- Make Feature

In [6]:
# # 송하인 기준으로 총 몇개의 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
# groupby_SEND_SPG_INNB_sum = data.groupby('SEND_SPG_INNB').sum()['INVC_CONT'].reset_index()
# # 이후에 피처로 만들어주기 위한 dic생성
# groupby_SEND_SPG_INNB_sum_dix = {x:y for x,y in zip(groupby_SEND_SPG_INNB_sum['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_sum['INVC_CONT'])}
# # 피처 생성
# data['총물류량_송하인'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_sum_dix[x])

In [7]:
# # 송하인 기준으로 총 몇명의 수하인에게 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
# groupby_SEND_SPG_INNB_count = data.groupby('SEND_SPG_INNB').count()['REC_SPG_INNB'].reset_index()
# # 이후에 피처로 만들어주기 위한 dic생성
# groupby_SEND_SPG_INNB_count_dix = {x:y for x,y in zip(groupby_SEND_SPG_INNB_count['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_count['REC_SPG_INNB'])}
# # 피처 생성
# data['거래수하인수'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_count_dix[x])

In [8]:
# # 평균 한건당 몇번의 수화물을 보내는지에 대한 피처 생성
# data['평균거래량_송하인'] = data['총물류량_송하인'] / data['거래수하인수']

In [9]:
# 수하인 기준으로 총 몇개의 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
groupby_REC_SPG_INNB_sum = data.groupby('REC_SPG_INNB').sum()['INVC_CONT'].reset_index()
# 이후에 피처로 만들어주기 위한 dic생성
groupby_REC_SPG_INNB_sum_dix = {x:y for x,y in zip(groupby_REC_SPG_INNB_sum['REC_SPG_INNB'], groupby_REC_SPG_INNB_sum['INVC_CONT'])}
# 피처 생성
data['총물류량_수하인'] = data['REC_SPG_INNB'].apply(lambda x: groupby_REC_SPG_INNB_sum_dix[x])

In [10]:
# 수하인 기준으로 총 몇명의 송하인에게 택배를 받았는지 확인하여 이를 피처로 만들어줌 
groupby_REC_SPG_INNB_count = data.groupby('REC_SPG_INNB').count()['SEND_SPG_INNB'].reset_index()
# 이후에 피처로 만들어주기 위한 dic생성
groupby_REC_SPG_INNB_count_dix = {x:y for x,y in zip(groupby_REC_SPG_INNB_count['REC_SPG_INNB'], groupby_REC_SPG_INNB_count['REC_SPG_INNB'])}
# 피처 생성
data['거래송하인수'] = data['REC_SPG_INNB'].apply(lambda x: groupby_REC_SPG_INNB_count_dix[x])

In [11]:
# 평균 한건당 몇번의 수화물을 보내는지에 대한 피처 생성
data['평균거래량_수하인'] = data['총물류량_수하인'] / data['거래송하인수']

In [12]:
# dic_max = {x: y for x, y in zip(data.groupby('SEND_SPG_INNB').max()[['DL_GD_LCLS_NM', '거래수하인수']].query('거래수하인수 > 100').reset_index()['SEND_SPG_INNB'], 
#                      data.groupby('SEND_SPG_INNB').max()[['DL_GD_LCLS_NM', '거래수하인수']].query('거래수하인수 > 100').reset_index()['DL_GD_LCLS_NM'])}
# data['최대거래품목'] = data['SEND_SPG_INNB'].apply(lambda x: dic_max[x] if x in dic_max else '거래량적음')

In [13]:
# # 상관관계가 높아서 제거
# data =  data.drop('거래수하인수', axis=1)

In [14]:
# 인코딩과 스케일링을 위함
data_index = data['index']
y_data = data['INVC_CONT']

data = data.drop(columns=['index', 'INVC_CONT'])

- 결측치 채움 -> 없음

- Encoding, Scailing

In [15]:
# # 수동으로 범주형 변수로 만들어줌
# data['SEND_SPG_INNB'] = data['SEND_SPG_INNB'].astype('str')
# data['REC_SPG_INNB'] = data['REC_SPG_INNB'].astype('str')

In [16]:
# 범주형 변수와 수치형 변수를 분리
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 

In [17]:
# scailing을 위해
num_features = num_features[2:]

In [18]:
#수치형변수 scailing
data[num_features] = PowerTransformer(standardize=True).fit_transform(data[num_features])

# # #라벨인코딩
# # #위의 범주형 변수들을 라벨인코딩
# for f in cat_features:
#     data[f] = LabelEncoder().fit_transform(data[f])

# # 원핫인코딩
# data = pd.get_dummies(data)

In [19]:
#위에서 떼어놨던 피처 병합
data = pd.concat([data_index, data],axis=1)
data = pd.concat([data, y_data],axis=1)

### PCA

In [20]:
# 차원축소 매소드 
from sklearn.decomposition import PCA

def dummy_to_pca(tr, column_name:str) :
    max_seq = 300
    max_d = 15
    col_count = tr.groupby(column_name)[column_name].count()
    if len(col_count) > max_seq:
        tops = col_count.sort_values(ascending=False)[0:max_seq].index
        f =tr.loc[tr[column_name].isin(tops)][['index', column_name]]
    else:
        tops = col_count.index
        f =tr[['index', column_name]]
    f = pd.get_dummies(f, columns=[column_name])  # This method performs One-hot-encoding
    f = f.groupby('index').mean()
    if len(tops) < max_d:
        max_d = len(tops)
    pca = PCA(n_components=max_d)
    pca.fit(f)
    cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합
    #print(cumsum)
    num_d = np.argmax(cumsum >= 0.99) + 1 # 분산의 설명량이 99%이상 되는 차원의 수
    if num_d == 1:
        num_d = max_d
    pca = PCA(n_components=num_d)    
    result = pca.fit_transform(f)
    result = pd.DataFrame(result)
    result.columns = [column_name + '_' + str(column) for column in result.columns]
    result.index = f.index
    return result.reset_index()

In [21]:
# PCA를 진행한 데이터 생성
pca = pd.concat([dummy_to_pca(data, 'DL_GD_LCLS_NM').iloc[:,1:], dummy_to_pca(data, 'DL_GD_MCLS_NM').iloc[:,1:]],axis=1)
data = pd.concat([data.drop(columns=['DL_GD_LCLS_NM', 'DL_GD_MCLS_NM']), pca], axis=1)

- Data split

In [22]:
#train과 test를 분리
train = data.iloc[:32000,:]
test = data.iloc[32000:,:]
test = test.drop(columns = ['INVC_CONT']).reset_index(drop=True)

In [23]:
train

,index,SEND_SPG_INNB,REC_SPG_INNB,총물류량_수하인,거래송하인수,평균거래량_수하인,INVC_CONT,DL_GD_LCLS_NM_0,DL_GD_LCLS_NM_1,DL_GD_LCLS_NM_2,DL_GD_LCLS_NM_3,DL_GD_LCLS_NM_4,DL_GD_MCLS_NM_0,DL_GD_MCLS_NM_1,DL_GD_MCLS_NM_2,DL_GD_MCLS_NM_3,DL_GD_MCLS_NM_4,DL_GD_MCLS_NM_5,DL_GD_MCLS_NM_6,DL_GD_MCLS_NM_7,DL_GD_MCLS_NM_8,DL_GD_MCLS_NM_9,DL_GD_MCLS_NM_10,DL_GD_MCLS_NM_11,DL_GD_MCLS_NM_12,DL_GD_MCLS_NM_13,DL_GD_MCLS_NM_14
0,0,1129000014045300,5011000220046300,0.392865,0.0,-0.233787,3.0,0.788362,0.287385,0.981666,-0.232391,-0.016447,0.676085,-0.024236,-0.068021,-0.009101,0.086673,-0.089317,0.018481,0.230694,-0.036317,-0.212550,-0.032277,0.036870,0.356156,0.849537,-0.172158
1,1,1135000009051200,5011000178037300,-0.285156,0.0,-0.327958,3.0,1.166577,-0.509271,-0.106340,-0.036534,-0.003234,0.678289,-0.024906,-0.070189,-0.009453,0.090753,-0.094908,0.019915,0.262160,-0.048143,-0.329765,-0.208902,0.773892,-0.362229,-0.094668,-0.120183
2,2,1135000030093100,5011000265091400,0.799279,0.0,-0.116074,9.0,0.788362,0.287385,0.981666,-0.232391,-0.016447,0.670055,-0.022555,-0.062661,-0.008250,0.077075,-0.076790,0.015408,0.173145,-0.021614,-0.107114,-0.009660,0.010137,0.054860,0.029683,0.871368
3,3,1154500002014200,5011000315087400,2.935636,0.0,3.250534,10.0,-0.152233,-0.009975,-0.005061,-0.002253,-0.000209,-0.317616,0.000973,0.002367,0.000261,-0.002049,0.001603,-0.000270,-0.002014,0.000154,0.000623,0.000040,-0.000040,-0.000187,-0.000088,-0.000412
4,4,1165000021008300,5011000177051200,1.483569,0.0,0.260610,3.0,-0.152233,-0.009975,-0.005061,-0.002253,-0.000209,0.739789,-0.084515,-0.374899,-0.701441,-0.452410,0.159187,-0.020342,-0.105791,0.006592,0.025245,0.001508,-0.001521,-0.006908,-0.003226,-0.013817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,31995,5011001060063300,2635000026053400,0.392865,0.0,-0.021903,6.0,-0.152233,-0.009975,-0.005061,-0.002253,-0.000209,-0.317616,0.000973,0.002367,0.000261,-0.002049,0.001603,-0.000270,-0.002014,0.000154,0.000623,0.000040,-0.000040,-0.000187,-0.000088,-0.000412
31996,31996,5011001095042400,1168000017002200,0.211131,0.0,0.401866,5.0,-0.152233,-0.009975,-0.005061,-0.002253,-0.000209,0.721631,-0.050784,-0.170036,-0.032908,0.684424,0.619441,-0.046337,-0.173290,0.009631,0.036001,0.002093,-0.002107,-0.009492,-0.004408,-0.018315
31997,31997,5011001108036200,4119700008012100,0.799279,0.0,-0.021903,9.0,-0.152233,-0.009975,-0.005061,-0.002253,-0.000209,-0.317616,0.000973,0.002367,0.000261,-0.002049,0.001603,-0.000270,-0.002014,0.000154,0.000623,0.000040,-0.000040,-0.000187,-0.000088,-0.000412
31998,31998,5011001115011400,1132000015085100,-0.285156,0.0,0.072268,3.0,-0.152233,-0.009975,-0.005061,-0.002253,-0.000209,-0.317616,0.000973,0.002367,0.000261,-0.002049,0.001603,-0.000270,-0.002014,0.000154,0.000623,0.000040,-0.000040,-0.000187,-0.000088,-0.000412


## Modeling with pycaret

- Model setting

In [24]:
# #pycaret에 넣도록 변형
# train = pd.concat([pd.DataFrame(train),y_train],axis=1)
# test = pd.concat([test_index, pd.DataFrame(test)],axis=1)

In [25]:
train = train.iloc[:,1:]

In [26]:
reg = setup(train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.8,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = 'INVC_CONT', # 목표 변수
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = True, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
#             numeric_features=['전체회차','현재회차'], # 숫자로써의 의미가 있다.
            session_id = 42,
            fold_shuffle = True
            )

,Description,Value
0,session_id,42
1,Target,INVC_CONT
2,Original Data,"(32000, 26)"
3,Missing Values,False
4,Numeric Features,25
5,Categorical Features,0
6,Transformed Train Set,"(25600, 25)"
7,Transformed Test Set,"(6400, 25)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [27]:
top5 = compare_models(sort='RMSE', fold=5, n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.2516,19.4858,4.3560,0.3824,0.2917,0.2169,0.8100
xgboost,Extreme Gradient Boosting,1.2110,20.2969,4.4860,0.3314,0.2927,0.2080,1.2380
rf,Random Forest Regressor,1.1733,20.6673,4.5183,0.3081,0.2915,0.1991,0.8400
catboost,CatBoost Regressor,1.2584,22.2012,4.6737,0.2686,0.2874,0.2212,3.8960
gbr,Gradient Boosting Regressor,1.3343,22.5998,4.7111,0.2400,0.2928,0.2334,1.5820
ridge,Ridge Regression,1.6461,24.0191,4.8096,0.2590,0.3578,0.3208,0.0100
omp,Orthogonal Matching Pursuit,1.6029,24.1470,4.8152,0.2592,0.3546,0.3135,0.0120
br,Bayesian Ridge,1.8533,24.8019,4.8905,0.2333,0.4027,0.3764,0.0200
et,Extra Trees Regressor,1.2202,25.3794,4.9927,0.1722,0.3162,0.2056,0.9520
lasso,Lasso Regression,1.8219,26.4813,5.0411,0.1886,0.3372,0.3422,0.6400


- Hyperparameter tuning

In [28]:
#lightgbm
lightgbm = create_model('lightgbm')
#hyperparameter tuning
lightgbm = tune_model(lightgbm, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 30)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.3451,14.8813,3.8576,0.4486,0.3176,0.2541
1,1.3407,15.1770,3.8958,0.4132,0.3130,0.2515
2,1.3127,14.3096,3.7828,0.4691,0.3102,0.2456
3,1.4403,31.7336,5.6333,0.4518,0.3093,0.2439
4,1.3504,17.7590,4.2141,0.2348,0.3143,0.2560
Mean,1.3578,18.7721,4.2767,0.4035,0.3129,0.2502
SD,0.0432,6.5879,0.6941,0.0863,0.0030,0.0047


In [28]:
#catboost
catboost = create_model('catboost')
#hyperparameter tuning
catboost = tune_model(catboost, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 30)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.9713,8.2329,2.8693,0.6950,0.2367,0.1721
1,0.9424,9.9569,3.1555,0.6150,0.2348,0.1639
2,0.9533,9.9542,3.1550,0.6307,0.2331,0.1648
3,1.0314,17.6340,4.1993,0.6954,0.2376,0.1693
4,0.9466,10.1614,3.1877,0.5621,0.2396,0.1676
Mean,0.9690,11.1879,3.3134,0.6396,0.2363,0.1675
SD,0.0327,3.2977,0.4578,0.0507,0.0022,0.0030


In [29]:
#gbr
gbr = create_model('gbr')
#hyperparameter tuning
gbr = tune_model(gbr, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 30)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.9250,9.5760,3.0945,0.6452,0.2379,0.1578
1,0.8949,10.2294,3.1983,0.6045,0.2340,0.1516
2,0.8887,9.0162,3.0027,0.6655,0.2250,0.1509
3,0.9848,18.4156,4.2913,0.6819,0.2375,0.1578
4,0.9237,12.7067,3.5646,0.4525,0.2382,0.1550
Mean,0.9234,11.9888,3.4303,0.6099,0.2345,0.1546
SD,0.0340,3.4517,0.4709,0.0829,0.0050,0.0029


In [65]:
# models = [
#     catboost,
#     gbr,
#     lightgbm,
#     ridge,
#     omp,
#     br
# ]

### Ensemble

- blend model

In [1]:
# final_model = blend_models(estimator_list = models, optimize = 'RMSE')
# #hyperparameter tuning
# final_model = tune_model(final_model, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 30)

In [30]:
final_model = finalize_model(lightgbm) # 전체 데이터로 재학습

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 

- Fit & predict

In [31]:
y_train = train['INVC_CONT']
train = train.drop('INVC_CONT', axis=1)

In [32]:
model = final_model
model.fit(train,y_train)
pred = model.predict(test.iloc[:,1:])

In [33]:
submission['INVC_CONT'] = pred

### Make submission 

In [34]:
t = pd.Timestamp.now()
fname = f'submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv'
# fname = 'submission_etpercentile.csv'
submission.to_csv('../submission/'+fname, index=False)
print("'{}' is ready to submit." .format(fname))

'submission_12161953.csv' is ready to submit.


In [35]:
beepsound()

# END